# Import Dependences

In [1]:
import os
# Find the latest version of spark 3.2.x  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.2.3'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 http://archive

In [2]:
# Install the PostgreSQL driver in our Colab environment
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-30 02:56:16--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.51MB/s    in 0.2s    

2022-11-30 02:56:16 (5.51 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Establish a Spark session and add the Postgres driver to the filepath
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AMZreviews").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract Data

In [4]:
# Read the first TSV.GZ file from an S3 bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
video_game_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
video_game_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [5]:
# Count the amount of rows in the dataset
print(f"Rows in dataset: {video_game_data_df.count()}")

Rows in dataset: 1785997


# Transform Data

In [6]:
# Count the amount of distinct rows in the dataset to find duplicates
print(f"Distinct rows in dataset: {video_game_data_df.distinct().count()}")

Distinct rows in dataset: 1785997


In [7]:
# Drop unnecessary columns
video_game_data_df = video_game_data_df.drop("marketplace", "product_category", "verified_purchase", "review_headline", "review_body")

video_game_data_df.show()

+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|
+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|          5|            0|          0|   N| 2015-08-31|
|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|          5|            0|          0|   N| 2015-08-31|
|    2331478|R3BH071QLH8QMC|B0029CSOD2|      98937668|Hidden Mysteries:...|          1|            0|          1|   N| 2015-08-31|
|   52495923|R127K9NTSXA2YH|B00GOOSV98|      23143350|GelTabz Performan...|          3|            0|          0|   N| 2015-08-31|
|   14533949|R32ZWUXDJPW27Q|B00Y074JOM|     821342511|Zero Suit Samus a...|        

In [8]:
# Find unique customer_id counts
print(f"Unique customer_id counts: {video_game_data_df.select('customer_id').distinct().count()}")

Unique customer_id counts: 1045733


In [9]:
# Find instances of each customer_id
customer_id_count_df = video_game_data_df.groupBy('customer_id').count()

customer_id_count_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   48670265|    1|
|   49103216|    2|
|    1131200|    1|
|   43076447|    2|
|   46261368|    1|
|    4883305|    5|
|   41192649|    1|
|   40985731|    7|
|   10437900|    2|
|   22245671|    1|
|    2574873|    1|
|    4696154|    1|
|    5621202|    1|
|    5871933|    2|
|   44089812|    1|
|    2845910|    1|
|    5274369|    1|
|   39069693|    2|
|     137793|    1|
|   31914942|    3|
+-----------+-----+
only showing top 20 rows



In [10]:
# Join count to original dataframe
video_game_data_df = video_game_data_df.join(customer_id_count_df, video_game_data_df.customer_id == customer_id_count_df.customer_id, "left").drop(video_game_data_df.customer_id)
video_game_data_df.show()

+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+-----------+-----+
|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|customer_id|count|
+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+-----------+-----+
|R1VTIA3JTYBY02|B00008KTNN|     384411423|Metal Arms: Glitc...|          5|            0|          0|   N| 2015-08-31|     106569|    1|
|R3BH071QLH8QMC|B0029CSOD2|      98937668|Hidden Mysteries:...|          1|            0|          1|   N| 2015-08-31|    2331478|    3|
|R3AQQ4YUKJWBA6|B002UBI6W6|     328764615|Psyclone Recharge...|          1|            0|          0|   N| 2015-08-31|    2377552|    1|
|R23H79DHOZTYAU|B0081EH12M|     770100932|New Trigger Grips...|          1|            1|          1|   N| 2015-08-31|    2963837|    2|
|R3IMF2MQ3OU9ZM|B002I0HIMI|     988218515

In [11]:
# Ensure rows are preserved
print(video_game_data_df.count())

1785997


In [12]:
# Rename count column
video_game_data_df = video_game_data_df.withColumnRenamed("count", "customer_count")
video_game_data_df.show()

+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+-----------+--------------+
|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|customer_id|customer_count|
+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+-----------+--------------+
|R1VTIA3JTYBY02|B00008KTNN|     384411423|Metal Arms: Glitc...|          5|            0|          0|   N| 2015-08-31|     106569|             1|
|R3BH071QLH8QMC|B0029CSOD2|      98937668|Hidden Mysteries:...|          1|            0|          1|   N| 2015-08-31|    2331478|             3|
|R3AQQ4YUKJWBA6|B002UBI6W6|     328764615|Psyclone Recharge...|          1|            0|          0|   N| 2015-08-31|    2377552|             1|
|R23H79DHOZTYAU|B0081EH12M|     770100932|New Trigger Grips...|          1|            1|          1|   N| 2015-08-31|    29

In [13]:
# Check data types
video_game_data_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- review_date: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = true)



In [27]:
from pyspark.sql.functions import to_date

# Cast customer_count to integer and review_date to date
video_game_data_df = video_game_data_df.withColumn("customer_count", video_game_data_df.customer_count.cast("integer"))
video_game_data_df = video_game_data_df.withColumn("review_date", video_game_data_df.review_date.cast("date"))

video_game_data_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- review_date: date (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = true)



In [28]:
# Create review_id dataframe for review_id_table in our database
review_id_df = video_game_data_df.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date']).dropDuplicates()
review_id_df.show() 

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1VTIA3JTYBY02|     106569|B00008KTNN|     384411423| 2015-08-31|
|R3BH071QLH8QMC|    2331478|B0029CSOD2|      98937668| 2015-08-31|
|R3AQQ4YUKJWBA6|    2377552|B002UBI6W6|     328764615| 2015-08-31|
|R23H79DHOZTYAU|    2963837|B0081EH12M|     770100932| 2015-08-31|
|R3IMF2MQ3OU9ZM|   10556786|B002I0HIMI|     988218515| 2015-08-31|
| RUL4H4XTTN2DY|   10712640|B00BUSLSAC|     829667834| 2015-08-31|
| RTIS3L2M1F5SM|   12039526|B001CXYMFS|     737716809| 2015-08-31|
|R32ZWUXDJPW27Q|   14533949|B00Y074JOM|     821342511| 2015-08-31|
| RYSSKR5ZTU8JX|   14533949|B0056WJA6C|     772407903| 2015-08-31|
|R3VNR804HYSMR6|   19676307|B00BRA9R6A|     682267517| 2015-08-31|
|R3UCNGYDVN24YB|   23091728|B002BSA388|      33706205| 2015-08-31|
| RIV24EQAIXA4O|   23092109|B005FMLZQQ|      24647669| 2015-08

In [36]:
# Create products dataframe for products in our database
products_df = video_game_data_df.select(['product_id', 'product_title']).dropDuplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00CJ7IUI6|The Elder Scrolls...|
|B00DHF39KS|Wolfenstein: The ...|
|B00MUTAVH6|Under Night In-Bi...|
|B001AZSEUW|              Peggle|
|B00KVOVBGM|PlayStation 4 Con...|
|B00O9VGH4Y|USPRO&reg; Headph...|
|B004OQNZY4|Phineas and Ferb:...|
|B00ZLN980O|Donop seablue 2.4...|
|B002L8W5V6|Dotop Nintendo Ga...|
|B007AJZ5PY|Nyko Game Case fo...|
|B000AOEU2K|Fire Emblem: Path...|
|B000H8BW7U|Tanarus (PC) (Com...|
|B013RADQOQ|Susenstone® 2400D...|
|B00KQXKUJ2|FIFA 15 (Ultimate...|
|B006W41X2C|Turtle Beach - Ea...|
|B000KCX9M4|Grand Theft Auto:...|
|B00YT90JWC|Red Wii Mini Cons...|
|B0096KG6A8|Wii U Super Mario...|
|B00L6AVLB0|World of Tanks-X3...|
|B000IMYKQ0|Wii Nunchuk Contr...|
+----------+--------------------+
only showing top 20 rows



In [37]:
# Create customers dataframe for customers in our database
customers_df = video_game_data_df.select(['customer_id', 'customer_count']).dropDuplicates()
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|      29729|             1|
|      32425|             1|
|      40514|             1|
|      45296|             1|
|     114975|             1|
|     142548|             1|
|     151483|             1|
|     205332|             1|
|     216607|             2|
|     218299|             1|
|     249348|             1|
|     336579|             5|
|     356945|             1|
|     359165|             1|
|     397913|             1|
|     416821|             1|
|     439110|             1|
|     452688|             1|
|     453816|             1|
|     462852|             1|
+-----------+--------------+
only showing top 20 rows



In [38]:
# Create vine dataframe for vine_table in our database
vine_df = video_game_data_df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine']).dropDuplicates()
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3W4IGRLNNQQPJ|          5|            1|          2|   N|
|R2FELCCWC4WUN3|          2|            1|          2|   N|
| RWYBKIHH0PW3G|          5|            0|          0|   N|
|R1PHVNSJTOBMWA|          4|            1|          2|   N|
|R335T8QW18YMBM|          1|            1|          6|   N|
|R15DME2AFFQ9OF|          5|            0|          1|   N|
|R29KPQZ5CZ6RLQ|          4|            0|          0|   N|
|R1BRJ8INCYO12T|          4|            0|          0|   N|
|R2K3WYZAIE1UTY|          5|            0|          0|   N|
|R2MWEZX26CSTMC|          5|            0|          0|   N|
| RQ49W9KYMN4JJ|          5|            0|          0|   N|
|R2M9SFYH6AJSJP|          5|            0|          0|   N|
|R28CK7LMIQSJ7L|          5|            0|          0|   N|
|R130QQG2TNMI6B|          5|            

# Load Data

In [32]:
# Fill out aws fields
my_aws_endpoint = ''
my_aws_port_number = '5432'
my_aws_database_name = 'amazon_big_data_db'
my_aws_username = 'postgres'
my_aws_password = ''

In [33]:
# Define the connection string
jdbc_url=f'jdbc:postgresql://{my_aws_endpoint}:{my_aws_port_number}/{my_aws_database_name}'

# Set up the configuration parameters
config = {"user": f'{my_aws_username}', 
          "password": f'{my_aws_password}', 
          "driver":"org.postgresql.Driver"}

# Choose to append the data
mode = 'append'

In [34]:
# Write the dataframe to the appropriate table in our PostgreSQL RDS

review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [39]:
# Write the dataframe to the appropriate table in our PostgreSQL RDS

products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [40]:
# Write the dataframe to the appropriate table in our PostgreSQL RDS

customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [41]:
# Write the dataframe to the appropriate table in our PostgreSQL RDS

vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)